# Simulation

In [1]:
# some necessary imports
import pandas as pd
import numpy as np
import plotly.plotly as py

# or functions are stored in utils module
from utils import init_veloc, trajectory, move_from_catch_to_new_origin, reverse_trajectory, plot_all, plot_traj

# Parameters

In [2]:
X_0 = 0 # initial x position in centimetres
Y_0 = 50 # initial height in centimetres
ALPHA = -np.pi/4 # fire angle (in radians) from x axis - MUST BE NEGATIVE
ENERGY_LOSS_RATIO = 0.7 # how much energy is lost because of bounce
J = 6.045 # http://scitation.aip.org/content/aapt/journal/tpt/50/5/10.1119/1.3703546
MASS = 2.456 
R = 1.970

# initial speed from bottle
LENGTH_OF_BOTTLE = 10 # cm
h = LENGTH_OF_BOTTLE * np.sin(-ALPHA)
# get initial speed of ball "poured" from bottle
V_0 = init_veloc(h, J, MASS, R)

# specify velocity angle under which we want to catch the ball
CATCH_ANGLE = -np.pi/4
VELOCITY_OF_ROBOT = 100  # speed of robot in cm/s

# Computation

In [3]:
# select time step and length
time_discret = np.arange(0,10,0.01)

# get the result for the bounce
bounce_traj, info = trajectory(time_discret, X_0, Y_0, V_0, ALPHA, ENERGY_LOSS_RATIO)
forward_traj = move_from_catch_to_new_origin(bounce_traj, CATCH_ANGLE, info["x_impact_point"], Y_0, VELOCITY_OF_ROBOT)
backward_traj = reverse_trajectory(forward_traj)

In [16]:
result_df = forward_traj.append(backward_traj)
result_df["time_diff"] = result_df.time.diff()
result_df["time_diff"].iloc[0] = 0
result_df[["time_diff", "x","y"]].to_csv("trajectory.csv", header=False, index=False)

# Plotting

In [4]:
py.iplot(plot_all(forward_traj, "Forward trajectory"), filename='pandas/forward')

In [5]:
py.iplot(plot_all(backward_traj, "Backward trajectory"), filename='pandas/backward')

In [6]:
# click on "Show closest data on hover" at top right corner to see times
py.iplot(plot_traj(forward_traj, backward_traj), filename='pandas/trajectory')

# Additional info

In [ ]:
info

The data for the first impact corresponds to data in this applet: http://www.walter-fendt.de/ph14e/projectile.htm .

The second shot can be obtained by following code and corresponds to data from e.g. here: https://phet.colorado.edu/sims/projectile-motion/projectile-motion_en.html

In [ ]:
rest = res[res.time >= info['impact_time']].copy()
rest.iloc[-1] - rest.iloc[0] 

Most info is from here:

* https://en.wikipedia.org/wiki/Trajectory_of_a_projectile
* https://en.wikipedia.org/wiki/Range_of_a_projectile
* https://en.wikipedia.org/wiki/Projectile_motion
* http://www.maththoughts.com/blog/2013/simple-physics-rolling/ - for ball speed when pouring from bottle

# Connection

In [ ]:
import socket

soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  
soc.connect(("127.0.0.1", 11991))

In [ ]:
import time

for ix, vals in res[['x','y']].iterrows():
    data = "{}\t{}".format(vals["x"],vals["y"])
    soc.sendall(data.encode("utf8"))

    if soc.recv(4096).decode("utf8") == "-":
        pass
    
    time.sleep(0.01)
    
soc.send(b'--ENDOFDATA--')